### opt main code

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import gurobipy as grb
from tqdm import tqdm
import matplotlib.pyplot as plt

# Please change the following path
cp = 'PATH'

# merge dataframe
## hazard scenario
df1 = pd.read_csv(cp + '/in/multihazard_all.csv',encoding='utf-8')
df1 = df1.fillna(0)
df1.loc[:, 'ID'] = df1['ID'].astype(str)

## Toyokawa levee breach scenario and available land
df = pd.read_csv(cp + '/in/multi_hazard.csv',encoding='utf-8')
df = df.fillna(0)
df.loc[:, 'ID'] = df['ID'].astype(str)

## Land price data
chika = pd.read_csv('../LandPrice/out/estimated_chika.csv',encoding = 'utf-8')
dfrt = chika.fillna(0)
dfrt.loc[:, 'ID'] = dfrt['ID'].astype(str)

## Merge levee breach scenario data, hazard scenario data, and land price data
df_merge = pd.merge(df, df1, left_on = 'ID', right_on = 'ID')
df_mergert = pd.merge(df_merge, dfrt, left_on = 'ID', right_on = 'ID')
df = df_mergert
df = df.drop_duplicates(subset=['ID']).reset_index(drop=True)
df = df.fillna(0)

## Add a Toyokawa hazard scenario 'equal'
df['equal'] = (df[['mh01', 'mh02', 'mh03', 'mh04', 'mh06', 'mh07']] == 1).any(axis=1).astype(int)

# Flood occurence probability pattern
dfh = pd.read_csv(cp + '/in/p_H.csv')
dfh = dfh.fillna(0)

# Probability p_s^B of 7 scenarios of levee breach
dfs = pd.read_csv(cp + '/in/p_sB.csv')
dfs = dfs.fillna(0)

# Effectiveness of proximity
dfe = pd.read_csv(cp + '/in/alpha.csv')

# original area data
df2 = pd.read_csv(cp + '/in/Toyohashi_area_x0_original.csv')
df2 = df2.fillna(0)

# A blanck data frame for optimised land use result
df3 = pd.read_csv(cp + '/in/opthere.csv')

## Dataframe for the output csv
cols = ['h','s','e','q','val.opt']
dfval = pd.DataFrame(index=[], columns=cols)

## index for grid row ID
Q = 0

# To Numpy
hazard = dfh.values
sB = dfs.values
effect = dfe.values

# Scenario Settings to run !! Please change the range accoring to which scenario you want to see
## Number of dataframe
number = df['ALAND']

# Hazard = range(0,len(hazard))
# Scenario = range(0,len(sB[0]))
# # Effect = range(0,len(effect))
# Effect = [0,5]

Hazard = [0,1,2,3]
Scenario = [0,1,2,3,4,5,6]
# Effect = range(0,len(effect))
Effect = [0,3,5]

## 
T = 30
I = range(0,len(number))
nK = 3
K = range(0,nK)

beta = 0.95

#management cost = 10,4919 JPY
# mc = [235405,50000,100000,200000,300000,400000,500000,600000,700000]
mc = [235405,50000,300000]

# Uncomment below if you want to calculate the value of objective function for original land use
#mc = [1000000000000]

# なぜこれが111でよいのかチェックする
alpha = [1,1,1]

# Delete the original land use area data if the grid is UNAVAILABLE area
# 5-15% areas will be deleted due to data consistency
df2['L'] = df2['NL']*df['ALAND']
df2['S'] = df2['NS']*df['ALAND']
df2['M'] = df2['NM']*df['ALAND']
dfm = df2.filter(items=['L','S','M'])
AA = dfm.values
d1 = dfm.iloc[:,0]
d2 = dfm.iloc[:,1]
d3 = dfm.iloc[:,2]

# Making dataframe of maximum areas per grid 
## Total areas = 100 m^2 
A = 10000 #MAX
Ava = df['ALAND']*A

## Total areas for each land use
D = [np.sum(d1),np.sum(d2),np.sum(d3)] #TOTAL

column_list = ['equal','mh01', 'mh02', 'mh03', 'mh04', 'mh06', 'mh07']
#column_list = ['umetagaw_1','yagyuuga_1','sanagawa_S','otowa_S_ne','kamita_S_s','sakai_S_ne','takasio_ne']

for q in [0]: # For each shrinking/expanding cost parameter settings

    ck = [48000,48000,48000]
    hk = [mc[q],mc[q],mc[q]]
    dk = [mc[q],mc[q],mc[q]]
    # ck = [45000,45000,45000]
    # hk = [5000,5000,100000]
    # dk = [5000,5000,100000]

    for h in Hazard: # For each flood occurence scenario 0, 0.03, 1
 
        for s in Scenario: # For each levee breach scenario
            h_flag = True
            #print(s)
            for e in Effect: # For each proximity effect scenario
                if h_flag: # A flag that represents if this is the first round of for e in Effect

                    # land price
                    df1 = df

                    # probability changed by scenario
                    H = hazard[h] # flood occurence probability
                    p = H*sB[0][s]*df1[column_list[s]]

                    Scenarios = []
                    # Pre-disaster land price
                    p_land_cols = ['if_jukyo', 'if_shogyo', 'if_kogyo']

                    # Post-disaster land price
                    for col in p_land_cols:
                        df1.loc[:, col + '_dmg'] = df1[col].copy()  # Copy Pre-
                        df1.loc[df1[column_list[s]] > 0, col + '_dmg'] = 0.8 * df1[col]  # Update to Post-

                    # New Land price dataframe
                    #df_0 = df1[[col for col in p_land_cols] + [col + '_dmg' for col in p_land_cols]]
                    df_0 = df1[[col for col in p_land_cols] + [col + '_dmg' for col in p_land_cols]].copy()
                    
                    name_s = 'exp_Lprice_' + column_list[s]
                    for col in p_land_cols:
                        df1[f'exp_Lprice_{column_list[s]}_{col}_{h}_{s}'] = ((1-pow(beta,T))/(1-beta))*df_0[col] - (df_0[col] - df_0[col + '_dmg'])*p*((1-pow(beta,T))/(1-beta)) - T*pow(beta,T)
                    temp_columns = [f'exp_Lprice_{column_list[s]}_{col}_{h}_{s}' for col in p_land_cols]
                    dfr = pd.concat([df1[temp_columns]], axis=1)
                    r = dfr.values

                    h_flag = False

                model = grb.Model('wcs')
                x,y,z,Z = {}, {}, {}, {}
                for i in I:
                    for k in K:
                        x[i,k] = model.addVar(vtype = 'C', lb = 0)
                for i in I:
                    for k in K:
                        y[i,k] = model.addVar(vtype = 'C', lb = 0)
                for i in I:
                    for k in K:
                        z[i,k] = model.addVar(vtype = 'C', lb = 0)
                #print(len(I))
                for i in I:
                    model.addConstr(grb.quicksum(x[i,k] for k in K) <= Ava[i])

                for k in K:
                    #print(x[i,k])
                    model.addConstr(grb.quicksum(x[i,k] for i in I) == D[k])
                df_net = pd.read_csv(cp + '/in/neighbourhood.csv', header = None)

                L = range(4)
                for i in I:
                    for k in K:
                        wa = 0
                        for l in L:
                            if df_net.iloc[i,l] >= 0:
                                a = df_net.iloc[i,l]
                                b = x[a,k]*effect[e]
                                wa = wa + b
                                Z[i,k] = wa
                for i in I:
                    for k in K:
                        model.addConstr(y[i,k] >= hk[k]*(x[i,k] - AA[i,k]))
                        model.addConstr(y[i,k] >= dk[k]*(-x[i,k] + AA[i,k]))
                        model.addConstr(z[i,k] <= alpha[k]*ck[k]*Z[i,k])
                        model.addConstr(z[i,k] - ck[k]*x[i,k] <= 0)

                model.setObjective(grb.quicksum(r[i,k]*x[i,k] for i in I for k in K) - grb.quicksum(ck[k]*x[i,k]-z[i,k]for i in I for k in K)  - grb.quicksum(y[i,k] for i in I for k in K), grb.GRB.MAXIMIZE)

                model.optimize()

                # Uncomment below for output Zij
                # cols = ['zij','i','j']
                # dfc = pd.DataFrame(index=[], columns=cols)
                # for k in K:
                #     dfc = pd.DataFrame(index=[], columns=cols)
                #     q = 0
                #     for i in I:
                #         wa = 0
                #         for l in L:
                #             if df_net.iloc[i,l] >= 0:
                #                  a = df_net.iloc[i,l]
                #                  if x[a,k].X > 0:
                #                      dfc.loc[q,'zij'] = x[a,k].X
                #                      dfc.loc[q,'i'] = i
                #                      dfc.loc[q,'j'] = a
                #                      q += 1
                #     dfc.to_csv(cp + '/out_01/Zij/Zij'+str(h)+str(s)+str(e)+str(k)+'.csv')

                df3.loc[I, 'optL'] = [x[i, 0].X for i in I]
                df3.loc[I, 'optS'] = [x[i, 1].X for i in I]
                df3.loc[I, 'optM'] = [x[i, 2].X for i in I]
                df3.loc[I, 'optL_diff'] = [AA[i,0]-x[i,0].X for i in I]
                df3.loc[I, 'optS_diff'] = [AA[i,1]-x[i,1].X for i in I]
                df3.loc[I, 'optM_diff'] = [AA[i,2]-x[i,2].X for i in I]
                
                # Output of the optimal value of original lanse use
                #df3.to_csv(cp + '/out_01/f/f9999.csv',index=False)

                # Output optimal land use
                df3.to_csv(cp + '/out_01/f/f' +str(h)+str(s)+str(e)+str(q)+'.csv',index=False)
                print(f'the file name is {h} {s} {e} {q}')

                val_opt = model.ObjVal
                dfval.loc[Q,'h'] = str(h)
                dfval.loc[Q,'s'] = str(s)
                dfval.loc[Q,'e'] = str(e)
                dfval.loc[Q,'q'] = str(q)
                dfval.loc[Q,'val.opt'] = val_opt
                Q += 1

# Output of the optimal value of original lanse use
#dfval.to_csv(cp +'/out_01/optvalorg.csv')

# Output optimal land use
dfval.to_csv(cp +'/out_01/optval.csv')

### Original Land Use
 - The original land use is calculated by setting a Big mc (demolish/counstruction cost) to avoid allocating areas to grids different from the current grids.
- mc = 10,000,000,000,000,000
- mc is supposed that it is not actually cost, so please be aware that the optimal allocations are exactly same as the original allocations.
- If there are differences between the original land use and the optimal land use, just sum the all differences.abs and multiply mc for calculating the cost 'that is not supposed to be cost'.

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import gurobipy as grb
from tqdm import tqdm
import matplotlib.pyplot as plt
import time

# Please change the following path
cp = 'PATH'

# merge dataframe
## hazard scenario
df1 = pd.read_csv(cp + '/in/multihazard_all.csv',encoding='utf-8')
df1 = df1.fillna(0)
df1.loc[:, 'ID'] = df1['ID'].astype(str)

## Toyokawa levee breach scenario and available land
df = pd.read_csv(cp + '/in/multi_hazard.csv',encoding='utf-8')
df = df.fillna(0)
df.loc[:, 'ID'] = df['ID'].astype(str)

## Land price data
chika = pd.read_csv('../LandPrice/out/estimated_chika.csv',encoding = 'utf-8')
dfrt = chika.fillna(0)
dfrt.loc[:, 'ID'] = dfrt['ID'].astype(str)

## Merge levee breach scenario data, hazard scenario data, and land price data
df_merge = pd.merge(df, df1, left_on = 'ID', right_on = 'ID')
df_mergert = pd.merge(df_merge, dfrt, left_on = 'ID', right_on = 'ID')
df = df_mergert
df = df.drop_duplicates(subset=['ID']).reset_index(drop=True)
df = df.fillna(0)

## Add a Toyokawa hazard scenario 'equal'
df['equal'] = (df[['mh01', 'mh02', 'mh03', 'mh04', 'mh06', 'mh07']] == 1).any(axis=1).astype(int)

# Flood occurence probability pattern
dfh = pd.read_csv(cp + '/in/p_H.csv')
dfh = dfh.fillna(0)

# Probability p_s^B of 7 scenarios of levee breach
dfs = pd.read_csv(cp + '/in/p_sB.csv')
dfs = dfs.fillna(0)

# Effectiveness of proximity
dfe = pd.read_csv(cp + '/in/alpha.csv')

# original area data
df2 = pd.read_csv(cp + '/in/Toyohashi_area_x0_original.csv')
df2 = df2.fillna(0)

# A blanck data frame for optimised land use result
df3 = pd.read_csv(cp + '/in/opthere.csv')

## Dataframe for the output csv
cols = ['h','s','e','q','val.opt']
dfval = pd.DataFrame(index=[], columns=cols)

## index for grid row ID
Q = 0

# To Numpy
hazard = dfh.values
sB = dfs.values
effect = dfe.values

# Scenario Settings to run !! Please change the range accoring to which scenario you want to see
## Number of dataframe
number = df['ALAND']

# Hazard = range(0,len(hazard))
# Scenario = range(0,len(sB[0]))
# # Effect = range(0,len(effect))
# Effect = [0,5]

# Hazard = [0,1,2,3]
# Scenario = [0,1,2,3,4,5,6]
# # Effect = range(0,len(effect))
# Effect = [0,3,5]

Hazard = [3]
Scenario = [0]
# Effect = range(0,len(effect))
Effect = [0]

## 
T = 30
I = range(0,len(number))
nK = 3
K = range(0,nK)

beta = 0.95

#management cost = 10,4919 JPY
# mc = [235405,50000,100000,200000,300000,400000,500000,600000,700000]
# mc = [235405,50000,300000]

# Uncomment below if you want to calculate the value of objective function for original land use
mc = [10000000000]

# なぜこれが111でよいのかチェックする
alpha = [1,1,1]

# Delete the original land use area data if the grid is UNAVAILABLE area
# 5-15% areas will be deleted due to data consistency
df2['L'] = df2['NL']*df['ALAND']
df2['S'] = df2['NS']*df['ALAND']
df2['M'] = df2['NM']*df['ALAND']
dfm = df2.filter(items=['L','S','M'])
AA = dfm.values
d1 = dfm.iloc[:,0]
d2 = dfm.iloc[:,1]
d3 = dfm.iloc[:,2]

# Making dataframe of maximum areas per grid 
## Total areas = 100 m^2 
A = 10000 #MAX
Ava = df['ALAND']*A

## Total areas for each land use
D = [np.sum(d1),np.sum(d2),np.sum(d3)] #TOTAL

column_list = ['equal','mh01', 'mh02', 'mh03', 'mh04', 'mh06', 'mh07']
#column_list = ['umetagaw_1','yagyuuga_1','sanagawa_S','otowa_S_ne','kamita_S_s','sakai_S_ne','takasio_ne']

start_time = time.time()
for q in [0]: # For each shrinking/expanding cost parameter settings

    ck = [48000,48000,48000]
    hk = [mc[q],mc[q],mc[q]]
    dk = [mc[q],mc[q],mc[q]]
    # ck = [45000,45000,45000]
    # hk = [5000,5000,100000]
    # dk = [5000,5000,100000]

    for h in Hazard: # For each flood occurence scenario 0, 0.03, 1
 
        for s in Scenario: # For each levee breach scenario
            h_flag = True
            #print(s)
            for e in Effect: # For each proximity effect scenario
                if h_flag: # A flag that represents if this is the first round of for e in Effect

                    # land price
                    df1 = df

                    # probability changed by scenario
                    H = hazard[h] # flood occurence probability
                    p = H*sB[0][s]*df1[column_list[s]]

                    Scenarios = []
                    # Pre-disaster land price
                    p_land_cols = ['if_jukyo', 'if_shogyo', 'if_kogyo']

                    # Post-disaster land price
                    for col in p_land_cols:
                        df1.loc[:, col + '_dmg'] = df1[col].copy()  # Copy Pre-
                        df1.loc[df1[column_list[s]] > 0, col + '_dmg'] = 0.8 * df1[col]  # Update to Post-

                    # New Land price dataframe
                    #df_0 = df1[[col for col in p_land_cols] + [col + '_dmg' for col in p_land_cols]]
                    df_0 = df1[[col for col in p_land_cols] + [col + '_dmg' for col in p_land_cols]].copy()
                    
                    name_s = 'exp_Lprice_' + column_list[s]
                    for col in p_land_cols:
                        df1[f'exp_Lprice_{column_list[s]}_{col}_{h}_{s}'] = ((1-pow(beta,T))/(1-beta))*df_0[col] - (df_0[col] - df_0[col + '_dmg'])*p*((1-pow(beta,T))/(1-beta)) - T*pow(beta,T)
                    temp_columns = [f'exp_Lprice_{column_list[s]}_{col}_{h}_{s}' for col in p_land_cols]
                    dfr = pd.concat([df1[temp_columns]], axis=1)
                    r = dfr.values

                    h_flag = False

                model = grb.Model('wcs')
                x,y,z,Z = {}, {}, {}, {}
                for i in I:
                    for k in K:
                        x[i,k] = model.addVar(vtype = 'C', lb = 0)
                for i in I:
                    for k in K:
                        y[i,k] = model.addVar(vtype = 'C', lb = 0)
                for i in I:
                    for k in K:
                        z[i,k] = model.addVar(vtype = 'C', lb = 0)
                #print(len(I))
                for i in I:
                    model.addConstr(grb.quicksum(x[i,k] for k in K) <= Ava[i])

                for k in K:
                    #print(x[i,k])
                    model.addConstr(grb.quicksum(x[i,k] for i in I) == D[k])
                df_net = pd.read_csv(cp + '/in/neighbourhood.csv', header = None)

                L = range(4)
                for i in I:
                    for k in K:
                        wa = 0
                        for l in L:
                            if df_net.iloc[i,l] >= 0:
                                a = df_net.iloc[i,l]
                                b = x[a,k]*effect[e]
                                wa = wa + b
                                Z[i,k] = wa
                for i in I:
                    for k in K:
                        model.addConstr(y[i,k] >= hk[k]*(x[i,k] - AA[i,k]))
                        model.addConstr(y[i,k] >= dk[k]*(-x[i,k] + AA[i,k]))
                        model.addConstr(z[i,k] <= alpha[k]*ck[k]*Z[i,k])
                        model.addConstr(z[i,k] - ck[k]*x[i,k] <= 0)

                model.setObjective(grb.quicksum(r[i,k]*x[i,k] for i in I for k in K) - grb.quicksum(ck[k]*x[i,k]-z[i,k]for i in I for k in K)  - grb.quicksum(y[i,k] for i in I for k in K), grb.GRB.MAXIMIZE)

                model.optimize()

                # Uncomment below for output Zij
                # cols = ['zij','i','j']
                # dfc = pd.DataFrame(index=[], columns=cols)
                # for k in K:
                #     dfc = pd.DataFrame(index=[], columns=cols)
                #     q = 0
                #     for i in I:
                #         wa = 0
                #         for l in L:
                #             if df_net.iloc[i,l] >= 0:
                #                  a = df_net.iloc[i,l]
                #                  if x[a,k].X > 0:
                #                      dfc.loc[q,'zij'] = x[a,k].X
                #                      dfc.loc[q,'i'] = i
                #                      dfc.loc[q,'j'] = a
                #                      q += 1
                #     dfc.to_csv(cp + '/out_01/Zij/Zij'+str(h)+str(s)+str(e)+str(k)+'.csv')

                # df3.loc[I, 'optL'] = [x[i, 0].X for i in I]
                # df3.loc[I, 'optS'] = [x[i, 1].X for i in I]
                # df3.loc[I, 'optM'] = [x[i, 2].X for i in I]
                # df3.loc[I, 'optL_diff'] = [AA[i,0]-x[i,0].X for i in I]
                # df3.loc[I, 'optS_diff'] = [AA[i,1]-x[i,1].X for i in I]
                # df3.loc[I, 'optM_diff'] = [AA[i,2]-x[i,2].X for i in I]
                sum_y = sum(y[i,k].X for i in I for k in K)
                                
                # Output of the optimal value of original lanse use
                #df3.to_csv(cp + '/out_01/f/f9999.csv',index=False)

                # Output optimal land use
                # df3.to_csv(cp + '/out_01/f/t' +str(h)+str(s)+str(e)+str(q)+'.csv',index=False)
                # print(f'the file name is {h} {s} {e} {q}')

                val_opt = model.ObjVal
                dfval.loc[Q,'h'] = str(h)
                dfval.loc[Q,'s'] = str(s)
                dfval.loc[Q,'e'] = str(e)
                dfval.loc[Q,'q'] = str(q)
                dfval.loc[Q,'val.opt'] = val_opt + sum_y
                Q += 1

# Output of the optimal value of original lanse use
#dfval.to_csv(cp +'/out_01/optvalorg.csv')
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Simulation time: {elapsed_time} seconds")

# # Output optimal land use
# dfval.to_csv(cp +'/out_01/optvalt.csv')